## 1 - Open source embedding model exploration

Exploring here other "open source" embedding models. 

For this task we will use MTEB that compares 100+ text (and image) embedding models across 1000, languages depending on metrics, languages, tasks, and task types:

https://huggingface.co/spaces/mteb/leaderboard

MTEB classe les modèles en fonction de  8 types de tâches  : ￼

1.	Bitext Mining : trouver des phrases correspondantes dans deux langues différentes.

2.	Classification : attribuer des catégories aux textes.

3.	Clustering : regrouper des textes similaires.

4.	Pair Classification : déterminer si deux textes sont similaires.

5.	Reranking : ordonner une liste de textes en fonction de leur pertinence par rapport à une requête.

6.	Retrieval : retrouver des documents pertinents pour une requête donnée.

7.	Semantic Textual Similarity (STS) : mesurer la similarité sémantique entre deux textes.

8.	Summarization : évaluer la qualité des résumés générés automatiquement. ￼


Critères de choix pour le modèle:
- Filtrer sur domaine juridique + langue Fr
- Performances en terme de Retrieval
- Nombre de tokens maximal à partir de 8191 en raison de la longeur de nos chunks (articles de loi dont la taille très variables)
- Taile du modele (en prenant le meilleur modele qui répond a tous les critères mais aussi un modele de 1B ou 0.5B)

Résultats de la selection en fonction des critères (classement en fonction des score sur Retreival):

- Catégorie 1 - modèles propriétaires: 

1 - [voyage-3](https://blog.voyageai.com/2024/09/18/voyage-3/) est le meilleur arrive en premier mais modele proprio (score de 85 en retreival) 

2 - ensuite vient celui de google [gemini-embedding-exp-03-07](https://developers.googleblog.com/en/gemini-embedding-text-model-now-available-gemini-api/) - (?B) -  max token 8192 - score global 74  - score retreival 67.71



- Catégorie 2 - modèles open source: 

1 - [inf-retriever-v1](https://huggingface.co/infly/inf-retriever-v1) (7B )  - score 73.89 mais aucun score sur STS ! sa version 1B est plus interessante car drop non significatif dans les perfs (72.14 )  [inf-retriever-v1-1.5b](https://huggingface.co/infly/inf-retriever-v1-1.5b)

2 - [SFR-Embedding-Mistral](https://huggingface.co/Salesforce/SFR-Embedding-Mistral) 7B  - max token 32768 - score 68.46

3 - [snowflake-arctic-embed-l-v2.0](https://huggingface.co/Snowflake/snowflake-arctic-embed-l-v2.0) - 0.5 B - max token 8192 - score 65

En intégrant le reranking au retreival : 

1 - [SFR-Embedding-Mistral](https://huggingface.co/Salesforce/SFR-Embedding-Mistral) - score global 84

2 - [snowflake-arctic-embed-l-v2.0](https://huggingface.co/Snowflake/snowflake-arctic-embed-l-v2.0) 80 en modele leger (0.5)


Remarque : Tous ces modèles semblent supérieurs aux perfs de [text-embedding-3-large](https://openai.com/index/new-embedding-models-and-api-updates/)  qui en retreival a un score de 59.27

In [ ]:
# 2. Sentence‑Transformers gère tout le reste
!pip install --upgrade sentence-transformers

In [ ]:
!pip install --upgrade pytorch

## 2 - Test loading inf-retriever-v1-1.5b ( on m2 mcbook air)

In [ ]:
import os
import torch
import numpy as np

In [11]:
import json, torch
from sentence_transformers import SentenceTransformer

# ----- 1) Choix de l'appareil (MPS pour Mac M1/M2) -----
device = "mps" if torch.backends.mps.is_available() else "cpu"


In [16]:
# ----- 2) Chargement du modèle prêt à l'emploi -----
model = SentenceTransformer(
    "infly/inf-retriever-v1-1.5b",   # ↙ modèle E5 + pooling
    device=device, 
    token=os.getenv("HF_token")
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [18]:
# ----- 3) Lecture du JSON source -----
with open("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_chunks/chunks_aml_5_strat_2.json", encoding="utf-8") as f:
    docs = json.load(f)

In [ ]:
# ----- 4) Génération des embeddings -----
for doc in docs:
    emb = model.encode(
        doc["page_content"],
        normalize_embeddings=True              # distance cosine déjà ok
    ).tolist()                                 # .tolist() pour JSON

    doc["embedding"] = emb                    # ajoute le vecteur
    # petite trace du modèle et de la dimension
    doc.setdefault("metadata", {})["embedding_model_name"] = (
        "infly/inf-retriever-v1-1.5b"
    )

In [ ]:
# ----- 5) Sauvegarde -----
with open("documents_with_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(docs, f, ensure_ascii=False, indent=2)

print("✅  Embeddings ajoutés dans documents_with_embeddings.json")

Conclusion : 
- Working : ok
- Result : None
- Taking time to execute.

## 2 - Testing the most efficient model : snowflake-arctic-embed-l-v2.0 (0.5 B)

 Even if this model is ranked at the 5th position it is the lighetest and most efficient one without a significant drop in performance (only 0.5 B param)

In [1]:
import os
import torch
import numpy as np

In [20]:
from sentence_transformers import SentenceTransformer

model_05B = SentenceTransformer(
    "Snowflake/snowflake-arctic-embed-l-v2.0",   # ↙ modèle E5 + pooling
    device=device, 
    token=os.getenv("HF_token")
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/251k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [21]:
# ----- 3) Lecture du JSON source -----
with open("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_chunks/chunks_aml_5_strat_2.json", encoding="utf-8") as f:
    docs = json.load(f)

In [ ]:
# ----- 4) Génération des embeddings -----
for doc in docs:
    emb = model.encode(
        doc["page_content"],
        normalize_embeddings=True              # distance cosine déjà ok
    ).tolist()                                 # .tolist() pour JSON

    doc["embedding"] = emb                    # ajoute le vecteur
    # petite trace du modèle et de la dimension
    doc.setdefault("metadata", {})["embedding_model_name"] = (
        "Snowflake-arctic-emb"
    )

In [23]:
# ----- 5) Sauvegarde -----
with open("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_chunks/snowflake-arctic_embedding_chunk_2.json", "w", encoding="utf-8") as f:
    json.dump(docs, f, ensure_ascii=False, indent=2)

print("✅  Embeddings ajoutés dans documents_with_embeddings.json")

✅  Embeddings ajoutés dans documents_with_embeddings.json
